# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Arthur"
# Set the data and output directories
DATA_ROOT = 'C:/Users/arthu/Downloads'
OUTPUT_ROOT = 'C:/Users/arthu/OneDrive/Documents/GitHub/output1'
TRAIT = 'Atherosclerosis'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [5]:
import os
from utils import *
# 1
dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

utils.py has been loaded


['TCGA_Colon_and_Rectal_Cancer_(COADREAD)']

If no match is found, jump directly to GEO in Part 2.2

In [8]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [24]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

NameError: name 'clinical_data_file' is not defined

In [ ]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [ ]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [ ]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [ ]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [ ]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [ ]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [ ]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [ ]:
gender_col = 'gender'

In [ ]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Lung-Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.639586,-0.195527,-0.324695,0.193167,-0.572812
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.997486,-0.276027,-1.219395,1.850967,-0.471212
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.959514,0.714373,0.112305,-0.532833,-0.003812
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,1.013722,-1.587117,0.359267,2.59051,0.806623,0.812114,0.367373,-0.494595,-1.062233,0.108588
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,0.110214,1.469373,0.194605,1.404667,-0.290712


In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [ ]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Lung-Cancer', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Lung-Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

A new JSON file was created at: ../../output1/Yuxuan/Lung-Cancer/cohort_info.json


## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [6]:
dataset = 'GEO'
trait_subdir = "Atherosclerosis"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE123086', 'GSE123088', 'GSE133601', 'GSE154851']

Repeat the below steps for all the accession numbers

In [25]:
cohort = accession_num = "GSE123088"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('C:/Users/arthu/Downloads\\GEO\\Atherosclerosis\\GSE123088\\GSE123088_family.soft.gz',
 'C:/Users/arthu/Downloads\\GEO\\Atherosclerosis\\GSE123088\\GSE123088_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [26]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"


In [27]:
clinical_data.head()

,!Sample_geo_accession,GSM3494884,GSM3494885,GSM3494886,GSM3494887,GSM3494888,GSM3494889,GSM3494890,GSM3494891,GSM3494892,...,GSM3495078,GSM3495079,GSM3495080,GSM3495081,GSM3495082,GSM3495083,GSM3495084,GSM3495085,GSM3495086,GSM3495087
0,!Sample_characteristics_ch1,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,...,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells,cell type: CD4+ T cells
1,!Sample_characteristics_ch1,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,primary diagnosis: ASTHMA,...,primary diagnosis: Breast cancer,primary diagnosis: Breast cancer,primary diagnosis: Breast cancer,primary diagnosis: Control,primary diagnosis: Breast cancer,primary diagnosis: Breast cancer,primary diagnosis: Breast cancer,primary diagnosis: Breast cancer,primary diagnosis: Control,primary diagnosis: Control
2,!Sample_characteristics_ch1,Sex: Male,diagnosis2: ATOPIC_ECZEMA,Sex: Female,Sex: Female,Sex: Male,Sex: Male,diagnosis2: ATHEROSCLEROSIS,Sex: Male,Sex: Female,...,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female,Sex: Female
3,!Sample_characteristics_ch1,age: 56,Sex: Male,age: 20,age: 51,age: 37,age: 61,Sex: Male,age: 31,age: 56,...,age: 78,age: 88,age: 41,age: 60,age: 72,age: 53,age: 73,age: 56,age: 38,age: 53
4,!Sample_characteristics_ch1,NaN,age: 63,NaN,NaN,NaN,NaN,age: 74,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['cell type: CD4+ T cells'],
 1: ['primary diagnosis: ASTHMA',
  'primary diagnosis: ATHEROSCLEROSIS',
  'primary diagnosis: BREAST_CANCER',
  'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA',
  'primary diagnosis: CROHN_DISEASE',
  'primary diagnosis: ATOPIC_ECZEMA',
  'primary diagnosis: HEALTHY_CONTROL',
  'primary diagnosis: INFLUENZA',
  'primary diagnosis: OBESITY',
  'primary diagnosis: PSORIASIS',
  'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS',
  'primary diagnosis: TYPE_1_DIABETES',
  'primary diagnosis: ACUTE_TONSILLITIS',
  'primary diagnosis: ULCERATIVE_COLITIS',
  'primary diagnosis: Breast cancer',
  'primary diagnosis: Control'],
 2: ['Sex: Male',
  'diagnosis2: ATOPIC_ECZEMA',
  'Sex: Female',
  'diagnosis2: ATHEROSCLEROSIS',
  'diagnosis2: ASTHMA_OBESITY',
  'diagnosis2: ASTHMA',
  'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS',
  'diagnosis2: OBESITY'],
 3: ['age: 56',
  'Sex: Male',
  'age: 20',
  'age: 51',
  'age: 37',
  'age: 61',
  'age: 31',
  'age: 41'

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [29]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Atherosclerosis\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Atherosclerosis\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key 

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [30]:
is_gene_availabe = False
trait_row = 1
age_row = 3 and 4
gender_row = 2 and 3

trait_type = 'binary'

In [31]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

This cohort is not usable. Please skip the following steps and jump to the next accession number.


In [14]:
# def convert_trait(tissue_type):
#     """
#     Convert tissue type to epilepsy presence (binary).
#     Assuming epilepsy presence for 'Hippocampus' tissue.
#     """
#     if tissue_type == 'primary diagnosis: ATHEROSCLEROSIS':
#         return 1  # Epilepsy present
#     else:
#         return 0  # Epilepsy not present

# def convert_age(age_string):
#     """
#     Convert age string to a numerical value.
#     """
#     if age_string.startswith('age: '):
#         try:
#             return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
#         except ValueError:
#             return None  # In case of parsing error
#     else:
#         return None  # If age string does not follow expected format

# def convert_gender(gender_string):
#     """
#     Convert gender string to a binary representation.
#     Female -> 0, Male -> 1, Unknown -> None
#     """
#     if gender_string == 'Sex: Female':
#         return 0
#     elif gender_string == 'Sex: Male':
#         return 1
#     else:
#         return None  # In case of unknown gender

# # Example usage
# example_tissue = 'tissue: Hippocampus'
# example_age = 'age: 23y'
# example_gender = 'gender: Male'

# converted_trait = convert_trait(example_tissue)
# converted_age = convert_age(example_age)
# converted_gender = convert_gender(example_gender)

# converted_trait, converted_age, converted_gender

# def convert_trait(value):
#     # Since Atherosclerosis is binary, return True if the value matches, otherwise return None
#     return True if value == 'primary diagnosis: ATHEROSCLEROSIS' else None

# def convert_age(value):
#     try:
#         # Attempt to convert the value to an integer
#         age = int(value.split(': ')[1])
#         return age
#     except:
#         # If conversion fails, return None
#         return None

# def convert_gender(value):
#     # If value is 'Male' or 'Female', return it, otherwise return None
#     return value.split(': ')[1] if value.split(': ')[1] in ['Male', 'Female'] else None

# def convert_trait(value):
#     # Binary trait (Atherosclerosis)
#     if value == 'ATHEROSCLEROSIS':
#         return 1
#     else:
#         return 0

# def convert_age(value):
#     # Continuous trait (Age)
#     try:
#         return int(value)
#     except (ValueError, TypeError):
#         return None

# def convert_gender(value):
#     # Binary trait (Gender)
#     if value == 'Male':
#         return 'Male'
#     elif value == 'Female':
#         return 'Female'
#     else:
#         return None
def convert_trait(value):
    if value.lower() == 'atherosclerosis':
        return True
    else:
        return False


def convert_age(value):
    # Check if the value is None or empty
    if value is None or not value.strip():
        return None
    
    # Remove any non-numeric characters from the value
    value = ''.join(filter(str.isdigit, value))
    
    # Check if the value is now empty
    if not value:
        return None
    
    # Convert the value to an integer
    age = int(value)
    
    # Determine the data type based on the range of ages
    if 0 <= age <= 120:  # Assuming realistic age range
        return age  # Continuous data type
    else:
        return None  # Unknown data type, return None

def convert_gender(value):
    if value is None:
        return None
    
    value = str(value).lower()  # Convert to lowercase for case-insensitive comparison
    if 'female' in value:
        return '0'
    elif 'male' in value:
        return '1'
    else:
        return None  # Return None for unknown or ambiguous values










In [15]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM4681537,GSM4681538,GSM4681539,GSM4681540,GSM4681541,GSM4681542,GSM4681543,GSM4681544,GSM4681545,GSM4681546,...,GSM4681597,GSM4681598,GSM4681599,GSM4681600,GSM4681601,GSM4681602,GSM4681603,GSM4681604,GSM4681605,GSM4681606
Atherosclerosis,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
Age,18,37,59,36,56,22,53,41,33,52,...,31,48,38,30,27,23,41,36,34,54
Gender,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### Genetic data preprocessing and final filtering

In [116]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM4681537,GSM4681538,GSM4681539,GSM4681540,GSM4681541,GSM4681542,GSM4681543,GSM4681544,GSM4681545,GSM4681546,...,GSM4681597,GSM4681598,GSM4681599,GSM4681600,GSM4681601,GSM4681602,GSM4681603,GSM4681604,GSM4681605,GSM4681606
ID,,,,,,,,,,,,,,,,,,,,,
1,4702.996309,4317.196744,1513.718847,1932.517654,18732.201180,15255.395810,10123.038710,9297.001384,869.165483,2803.75233,...,72390.580520,73905.387240,62863.701400,66163.108640,53456.572790,60633.859790,71472.691040,61301.248940,58363.452640,44245.405160
2,50.312383,56.696744,58.718847,60.517654,66.201178,62.395814,77.038713,57.501384,68.665483,64.75233,...,65.580515,60.387238,66.701397,63.108644,71.572793,63.859788,69.691041,69.748941,86.452639,68.405155
3,55.996309,55.696744,54.718856,54.018310,65.201178,62.395814,66.038752,65.001384,71.165483,67.75233,...,67.580515,67.387238,61.701397,64.108644,61.072793,69.859788,74.691041,76.748941,74.452639,81.905155
4,2114.996309,1025.696744,2267.718847,1661.017654,3390.201178,2211.395814,1447.038713,3201.501384,2660.665483,2058.75233,...,2397.580515,2270.887238,2426.701397,2438.608644,1228.072793,2142.859788,2155.691041,2757.748941,2269.452639,2155.405155
5,58.996309,52.707022,56.718847,51.709381,58.201178,62.395814,88.538713,62.501384,67.165483,67.75233,...,86.080515,71.887238,74.701397,75.608644,78.072793,71.859788,70.691041,83.748941,72.952639,72.405155


In [117]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [118]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20']\n"

If not required, jump directly to the gene normalization step

In [119]:
requires_gene_mapping = True

In [120]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'heme binding protein 1', 'potassium voltage-gated channel, Isk-related family, member 4'], 'UNIGENE_ID': [nan, nan, nan, 'Hs.642618', 'Hs.348522'], 'ENSEMBL_ID': [nan, nan, nan, 'ENST00000014930', 'ENST00000281830'], 'ACCESSION_STRING': [nan, nan, nan, 'ref|NM_015987|ens|ENST00000014930|gb|AF117615|gb|BC016277', 'ref|NM_080671|ens|ENST00000281830|tc|THC2655788'], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, 'chr12:13

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [121]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'NAME': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P117082', 'A_33_P3246448'], 'SPOT_ID': ['CONTROL', 'CONTROL', 'CONTROL', 'A_23_P117082', 'A_33_P3246448'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'GB_ACC': [nan, nan, nan, 'NM_015987', 'NM_080671'], 'LOCUSLINK_ID': [nan, nan, nan, 50865.0, 23704.0], 'GENE_SYMBOL': [nan, nan, nan, 'HEBP1', 'KCNE4'], 'GENE_NAME': [nan, nan, nan, 'he

In [123]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [124]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [125]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [126]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 70 samples.


In [127]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Atherosclerosis', the least common label is 'False' with 70 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Atherosclerosis' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 26.0
  50% (Median): 33.0
  75%: 41.0
Min: 18
Max: 71
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is 'male' with 2 occurrences. This represents 2.86% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



True

In [128]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [129]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [26]:
from utils import *

In [27]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'GSE123088'

In [28]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
1,GSE123088,True,True,False,True,True,192,
0,GSE123086,True,True,False,True,True,154,


In [30]:
ck=100
listq=[]
for df in pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'),chunksize=ck):
    listq.append(df)
merged_data = pd.concat(listq)
#merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [31]:
merged_data.head()

,Atherosclerosis,Age,Gender
0,0.0,56.0,1.0
1,0.0,20.0,0.0
2,0.0,51.0,0.0
3,0.0,37.0,1.0
4,0.0,61.0,1.0


In [29]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [30]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [31]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

c:\Users\arthu\OneDrive\Documents\GitHub\TAIS_2024SP\Arthur\..\utils.py:631: RuntimeWarning: divide by zero encountered in divide
  normalized_ev = eigen_values / eigen_values[0]
c:\Users\arthu\OneDrive\Documents\GitHub\TAIS_2024SP\Arthur\..\utils.py:631: RuntimeWarning: invalid value encountered in divide
  normalized_ev = eigen_values / eigen_values[0]


False

In [32]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [33]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

ValueError: Found array with 0 feature(s) (shape=(154, 0)) while a minimum of 1 is required by Lasso.

In [37]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

AttributeError: Module 'scipy' has no attribute 'exp'

In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0025
Found 547 significant genes associated with the trait 'Psoriasis' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient  corrected_p_value
4.430015695     0.087750           0.001231
4.478193647     0.087750           0.001231
4.478966158     0.087750           0.001231
 4.48687579     0.087750           0.001231
4.491851159     0.087750           0.001231
4.492453853     0.087750           0.001231
4.494270678     0.087750           0.001231
4.504245863     0.087750           0.001231
4.505190735     0.087750           0.001231
4.505235806     0.087750           0.001231
4.506891502     0.087750           0.001231
4.510669146     0.087750           0.001231
4.476613942     0.087750           0.001231
 4.51198966     0.087750           0.001231
4.513528153     0.087750           0.001231
4.516930407     0.087750           0.001231
4.806825082     0.087750           0.001

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/utils.py:425: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values


In [ ]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)